In [22]:
import torch
import torch.nn as nn
import os

from data_preprocessing import get_files, labels_distribution
from train_test import create_iterator, run_train, evaluate
from lstm_model import LSTM

if __name__ == "__main__":

    # placing the tensors on the GPU if one is available.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False

    # parameters
    model_type = "LSTM"
    data_type = "token" # or: "morph"

    char_based = True
    if char_based:
        tokenizer = lambda s: list(s) # char-based
    else:
        tokenizer = lambda s: s.split() # word-based


    # hyper-parameters:
    lr = 1e-4
    batch_size = 50
    dropout_keep_prob = 0.5
    embedding_size = 300
    max_document_length = 100  # each sentence has until 100 words
    dev_size = 0.8 # split percentage to train\validation data
    max_size = 5000 # maximum vocabulary size
    seed = 1
    num_classes = 3

    # dropout_keep_prob, embedding_size, batch_size, lr, dev_size, vocabulary_size, max_document_length, input_size, hidden_size, output_dim, n_filters, filter_sizes, num_epochs = get_params(model_type)
    train_data, valid_data, test_data, Text, Label = get_files("", dev_size, max_document_length, seed, data_type, tokenizer)

    # Build_vocab : It will first create a dictionary mapping all the unique words present in the train_data to an
    # index and then after it will use word embedding (random, Glove etc.) to map the index to the corresponding word embedding.
    Text.build_vocab(train_data, max_size=max_size)
    Label.build_vocab(train_data)
    vocab_size = len(Text.vocab)

    train_iterator, valid_iterator, test_iterator = create_iterator(train_data, valid_data, test_data, batch_size, device)

    # loss function
    loss_func = nn.CrossEntropyLoss()

    if (model_type == "LSTM"):

        num_hidden_nodes = 93
        hidden_dim2 = 128
        num_layers = 2  # LSTM layers
        bi_directional = False
        num_epochs = 7

        to_train = True
        pad_index = Text.vocab.stoi[Text.pad_token]

        # Build the model
        lstm_model = LSTM(vocab_size, embedding_size, num_hidden_nodes, hidden_dim2 , num_classes, num_layers,
                       bi_directional, dropout_keep_prob, pad_index)

        # optimization algorithm
        optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)

        # train and evaluation
        if (to_train):
            # train and evaluation
            run_train(num_epochs, lstm_model, train_iterator, valid_iterator, optimizer, loss_func, model_type)

            # load weights
        lstm_model.load_state_dict(torch.load("saved_weights_LSTM.pt"))
        # predict
        test_loss, test_acc = evaluate(lstm_model, test_iterator, loss_func)
        print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

Number of training examples: 8195
Number of validation examples: 2049
Number of testing examples: 2560
	Train Loss: 0.836 | Train Acc: 63.51%
	 Val. Loss: 0.745 |  Val. Acc: 65.08%
	Train Loss: 0.724 | Train Acc: 66.84%
	 Val. Loss: 0.729 |  Val. Acc: 65.08%
	Train Loss: 0.704 | Train Acc: 66.84%
	 Val. Loss: 0.707 |  Val. Acc: 65.13%
	Train Loss: 0.673 | Train Acc: 67.58%
	 Val. Loss: 0.646 |  Val. Acc: 68.21%
	Train Loss: 0.625 | Train Acc: 70.37%
	 Val. Loss: 0.603 |  Val. Acc: 70.13%
	Train Loss: 0.591 | Train Acc: 72.01%
	 Val. Loss: 0.589 |  Val. Acc: 72.58%
	Train Loss: 0.568 | Train Acc: 73.69%
	 Val. Loss: 0.573 |  Val. Acc: 74.28%


NameError: name 'path' is not defined

In [29]:
train_data